# Creating Star Schema 

This notebook explores the creation of a Star Schema from raw data using Apache Spark.

The data used in this notebook is from the Brazilian Basic Education Census, which is available at the [Brazilian government's open data portal](https://download.inep.gov.br/dados_abertos)

## Creating SparkSession

The first step is to create a SparkSession.

In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

import psycopg2
from datetime import datetime

from itertools import chain

In [2]:
spark = SparkSession.builder\
        .appName("CensoEscolarStarSchema")\
        .config("spark.sql.shuffle.partitions", "4")\
        .getOrCreate()

/home/joaopedro/.local/lib/python3.6/site-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


## Transforming CSV to Parquet

This transformation aims to increase the speed of data loading by using Parquet files.

In [ ]:
# Read CSV data
data_csv = (
    spark
    .read
    .format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .option("delimiter", ";")
    .option("encoding", "latin1")
    .load("./data/*.csv")
)

Write to Parquet file

In [6]:
data_csv.write.parquet("./data/censo_escolar.parquet")

Reading from Parquet file

In [3]:
data = (
    spark
    .read
    .format("parquet")
    .load("./data/censo_escolar.parquet/")
)

## Dimensions

The dimensions are...

---

The code below creates the dimensions based on a configuration dict.

```json
{
    "DIM_NAME":{
        # The fields are the table columns
        "fields":[
            {
                "field":"FIELD_1_NAME", # The column name
                "type":"FIELD_1_TYPE",  # The column type in spark
            },
            {
                "field":"FIELD_2_NAME",
                "type":"FIELD_2_TYPE",
            },
            ...
        ],
    }
}
```

In [2]:
INTEGER_DIMENSIONS = [
    "TP_DEPENDENCIA",
    "TP_LOCALIZACAO",
    "IN_AGUA_POTAVEL",
    "IN_ENERGIA_INEXISTENTE",
    "IN_ESGOTO_INEXISTENTE",
    "IN_BANHEIRO",
    "IN_BIBLIOTECA",
    "IN_REFEITORIO",
    "IN_COMPUTADOR",
    "IN_INTERNET",
    "IN_EQUIP_NENHUM"
]

DIMENSION_TABLES_CONFIG = {
    "DIM_LOCAL":{
        "fields": [
            {"field":"NO_UF", "type":"string",},
            {"field":"SG_UF", "type":"string",},
            {"field":"CO_UF", "type":"string",},
            {"field":"NO_MUNICIPIO", "type":"string",},
            {"field":"CO_MUNICIPIO", "type":"string",}
        ]
    },
}

DIMENSION_TABLES_CONFIG.update(
    {
        "DIM_"+dimension.upper():{
            "fields": [
                {"field":dimension, "type":"integer"} 
            ]
        }
        for dimension in INTEGER_DIMENSIONS
    }
)

### Creating dimensions table in Postgres
-----------------------------------------------------------------------------------------------------------------------

Defining the properties of the Postgres Connection

In [5]:
POSTGRES_USER = "censo"
POSTGRES_PASSWORD = "123"
POSTGRES_DB = "censo_escolar"

# Used to connect to the PostgreSQL database server
# in spark session
POSTGRES_CONFIG = {
    "url":f"jdbc:postgresql://localhost:5432/{POSTGRES_DB}",
    "properties":{
        "user":POSTGRES_USER, 
        "password":POSTGRES_PASSWORD,
        "driver":"org.postgresql.Driver",
    },
}

Establishing connection to Postgres

In [6]:
conn = psycopg2.connect(
    host="localhost",
    port="5432",

    dbname=POSTGRES_DB,
    user=POSTGRES_USER,
    password=POSTGRES_PASSWORD
)

Function to create a Dimension table in Postgres using the configuration in DIMENSION_TABLES_CONFIG and adding an id column

The code below creates the dimensions
Spark will create a table with the name of the dimension and the columns in the configuration

In [7]:
# Write data to Postgres
# Using the configuration in DIMENSION_TABLES_CONFIG
# With id as the primary key

for table_name, table_config in DIMENSION_TABLES_CONFIG.items():
    
    print(f"[{datetime.now()}] Writing {table_name}")
    
    data\
    .select(
        [
            F
            .col(field["field"])
            .cast(field["type"])
            .alias(field["field"])
            
            for field
            in table_config["fields"]
        ]
    )\
    .distinct()\
    .withColumn(
        "id", F.monotonically_increasing_id()
    )\
    .write\
    .jdbc(
        **POSTGRES_CONFIG,
        table=table_name,
        mode="overwrite"
    )
    
    print(f"[{datetime.now()}] Wrote {table_name}")
    # Define id as the primary key
    cursor = conn.cursor()
    cursor.execute(
        f"ALTER TABLE {table_name} ADD PRIMARY KEY (id);"
    )
    cursor.close()
    conn.commit()

    print(f"[{datetime.now()}] Added primary key to {table_name}")
    print(f"[{datetime.now()}] Done")

[2022-07-25 02:01:51.073109] Writing DIM_LOCAL
[2022-07-25 02:02:00.865734] Wrote DIM_LOCAL
[2022-07-25 02:02:00.889941] Added primary key to DIM_LOCAL
[2022-07-25 02:02:00.890092] Done
[2022-07-25 02:02:00.890113] Writing DIM_TP_DEPENDENCIA
[2022-07-25 02:02:02.179984] Wrote DIM_TP_DEPENDENCIA
[2022-07-25 02:02:02.188489] Added primary key to DIM_TP_DEPENDENCIA
[2022-07-25 02:02:02.188640] Done
[2022-07-25 02:02:02.188660] Writing DIM_TP_LOCALIZACAO
[2022-07-25 02:02:03.262895] Wrote DIM_TP_LOCALIZACAO
[2022-07-25 02:02:03.275347] Added primary key to DIM_TP_LOCALIZACAO
[2022-07-25 02:02:03.276650] Done
[2022-07-25 02:02:03.276778] Writing DIM_IN_AGUA_POTAVEL
[2022-07-25 02:02:04.478306] Wrote DIM_IN_AGUA_POTAVEL
[2022-07-25 02:02:04.492452] Added primary key to DIM_IN_AGUA_POTAVEL
[2022-07-25 02:02:04.492676] Done
[2022-07-25 02:02:04.492703] Writing DIM_IN_ENERGIA_INEXISTENTE
[2022-07-25 02:02:05.313743] Wrote DIM_IN_ENERGIA_INEXISTENTE
[2022-07-25 02:02:05.342052] Added primary key

## Facts table

The definition of the facts table follows a different pattern than the dimensions.

The table schema is previously defined to properly define the foreing keys.


Defining the facts table schema
Metrics + Facts + Dimensions (Foreign Keys)

In [3]:
FACT_TABLE_NAME = "FACT_CENSO_ESCOLAR"

FACT_COLUMNS = [
    "QT_DOC_BAS",	# Número de Docentes da Educação Básica
    "QT_DOC_INF",	# Número de Docentes da Educação Infantil
    "QT_DOC_FUND",	# Número de Docentes do Ensino Fundamental
    "QT_DOC_MED",	# Número de Docentes do Ensino Médio

    "QT_MAT_BAS",	# Número de Matrículas na Educação Básica (TOTAL)
    "QT_MAT_INF",	# Número de Matrículas na Educação Infantil
    "QT_MAT_FUND",	# Número de Matrículas no Ensino Fundamental
    "QT_MAT_MED",	# Número de Matrículas no Ensino Médio

    "QT_MAT_BAS_ND",	    # Número de Matrículas na Educação Básica - Cor/Raça Não Declarada
    "QT_MAT_BAS_BRANCA",	# Número de Matrículas na Educação Básica - Cor/Raça Branca
    "QT_MAT_BAS_PRETA",	    # Número de Matrículas na Educação Básica - Cor/Raça Preta
    "QT_MAT_BAS_PARDA",	    # Número de Matrículas na Educação Básica - Cor/Raça Parda
    "QT_MAT_BAS_AMARELA",	# Número de Matrículas na Educação Básica - Cor/Raça Amarela
    "QT_MAT_BAS_INDIGENA",	# Número de Matrículas na Educação Básica - Cor/Raça Indígena
    
    "NU_ANO_CENSO"
]

FACT_CONFIG = {
    fact:{
        "fields": [
            {"field":fact, "type":"integer"}
        ]
    }
    for fact in FACT_COLUMNS
}

DIMENSION_ID_CONFIG = {
    table_name:[
        field['field'] 
        for field 
        in table_fields['fields']
    ]
    for table_name, table_fields in DIMENSION_TABLES_CONFIG.items()
}

FACT_TABLE_ALL_COLUMNS_ORDERED = FACT_COLUMNS + list(map(lambda col:"ID_"+col, DIMENSION_ID_CONFIG.keys()))

Before inserting the data into the facts table, we need to create a function to create the facts table in Postgres

The code below creates the facts table in Postgres using the configuration in FACT_TABLES_CONFIG and adding an id column for each dimension

In [4]:
# Create fact table
# Using the configuration in FACT_CONFIG
# With id as the primary key

# Avoid inserting a backslash into a f-string
comma_break_line = ",\n\t\t\t"
facts_table_sql = f"""
    CREATE TABLE IF NOT EXISTS {FACT_TABLE_NAME} (
        id SERIAL PRIMARY KEY,
        { 
            comma_break_line.join(
                [
                    f"{field} INTEGER" 
                    for field in FACT_COLUMNS
                ]
                +[
                    f"ID_{dim_table} BIGINT"
                    for dim_table in DIMENSION_ID_CONFIG.keys()
                ]
            )
        }
    );
    
    -- Adding Foreign Keys
    ALTER TABLE {FACT_TABLE_NAME}
    {
        comma_break_line.join(
            [
                f"ADD CONSTRAINT {FACT_TABLE_NAME}_{dim_table}_fk FOREIGN KEY (ID_{dim_table}) REFERENCES {dim_table}(id)"
                for dim_table in DIMENSION_ID_CONFIG.keys()
            ]
        )
    }
"""

In [6]:
print(facts_table_sql)


    CREATE TABLE IF NOT EXISTS FACT_CENSO_ESCOLAR (
        id SERIAL PRIMARY KEY,
        QT_DOC_BAS INTEGER,
			QT_DOC_INF INTEGER,
			QT_DOC_FUND INTEGER,
			QT_DOC_MED INTEGER,
			QT_MAT_BAS INTEGER,
			QT_MAT_INF INTEGER,
			QT_MAT_FUND INTEGER,
			QT_MAT_MED INTEGER,
			QT_MAT_BAS_ND INTEGER,
			QT_MAT_BAS_BRANCA INTEGER,
			QT_MAT_BAS_PRETA INTEGER,
			QT_MAT_BAS_PARDA INTEGER,
			QT_MAT_BAS_AMARELA INTEGER,
			QT_MAT_BAS_INDIGENA INTEGER,
			NU_ANO_CENSO INTEGER,
			ID_DIM_LOCAL BIGINT,
			ID_DIM_TP_DEPENDENCIA BIGINT,
			ID_DIM_TP_LOCALIZACAO BIGINT,
			ID_DIM_IN_AGUA_POTAVEL BIGINT,
			ID_DIM_IN_ENERGIA_INEXISTENTE BIGINT,
			ID_DIM_IN_ESGOTO_INEXISTENTE BIGINT,
			ID_DIM_IN_BANHEIRO BIGINT,
			ID_DIM_IN_BIBLIOTECA BIGINT,
			ID_DIM_IN_REFEITORIO BIGINT,
			ID_DIM_IN_COMPUTADOR BIGINT,
			ID_DIM_IN_INTERNET BIGINT,
			ID_DIM_IN_EQUIP_NENHUM BIGINT
    );
    
    -- Adding Foreign Keys
    ALTER TABLE FACT_CENSO_ESCOLAR
    ADD CONSTRAINT FACT_CENSO_ESCOLAR_DIM_LOCAL_fk FORE

Executing the function to create the facts table in Postgres

In [10]:
print(f"[{datetime.now()}] Creating facts table")

cursor = conn.cursor()
try:
    cursor.execute(facts_table_sql)
    cursor.close()
    conn.commit()
except Exception as e:
    print(e)
    conn.rollback()
    cursor.close()
else:
    print(f"[{datetime.now()}] Created facts table")
    print(f"[{datetime.now()}] Done")


[2022-07-25 02:02:11.470506] Creating facts table
[2022-07-25 02:02:11.493910] Created facts table
[2022-07-25 02:02:11.494028] Done


Extracting the data

In [11]:
facts_data = data\
    .select(
        [
            *chain(
                *DIMENSION_ID_CONFIG.values(), 
                FACT_CONFIG.keys()
            )
        ]
    )

Adding the ids for each dimension

In [12]:
# Joining the id of the dimensions

for table_name, table_fields in DIMENSION_ID_CONFIG.items():
    
    # Read the dimension data from Postgres
    dim_table = spark.read\
        .jdbc(
            **POSTGRES_CONFIG,
            table=table_name,
        )\
        .withColumnRenamed("id", f"ID_{table_name}")
    
    # Join the dimension data with the fact data
    facts_data = facts_data\
        .join(
            dim_table,
            on=table_fields,
            how="left"
        )\
        .drop(*table_fields)

Saving data to Postgres

In [14]:
# Order the columns to match the fact table on postgres
# and save the data
facts_data\
    .select(*FACT_TABLE_ALL_COLUMNS_ORDERED)\
    .write\
    .jdbc(
        **POSTGRES_CONFIG,
        table=FACT_TABLE_NAME,
        mode="append"
    )

## References

https://towardsdatascience.com/explaining-technical-stuff-in-a-non-techincal-way-apache-spark-274d6c9f70e9

https://towardsdatascience.com/adding-sequential-ids-to-a-spark-dataframe-fa0df5566ff6

https://sparkbyexamples.com/pyspark/pyspark-read-and-write-parquet-file/

https://www.psycopg.org/docs/usage.html
